In [7]:
import sys

from collections import deque

# Константы для символов ключей и дверей
keys_char = [chr(i) for i in range(ord('a'), ord('z') + 1)]
doors_char = [k.upper() for k in keys_char]

def get_input():
    """Чтение данных из стандартного ввода."""
    return [list(line.strip()) for line in sys.stdin]

def solve(data): #решение создано с помощью чата gpt, никак не могла дойти до того, как возвращаться в роботам несколько раз
    rows = len(data)
    cols = len(data[0]) if rows > 0 else 0
    
    # Найти стартовые позиции роботов и количество ключей
    robots = []
    key_count = 0
    key_positions = {}
    
    for r in range(rows):
        for c in range(cols):
            if data[r][c] == '@':
                robots.append((r, c))
            elif 'a' <= data[r][c] <= 'z':
                key_positions[data[r][c]] = (r, c)
                key_count += 1

    # Всех ключей по порядку
    keys = list(key_positions.keys())

    # Функция для BFS поиска ключей
    def bfs(robot_start):
        queue = deque([robot_start])
        visited = set([robot_start])
        distances = {}
        while queue:
            r, c = queue.popleft()
            if data[r][c] in key_positions:
                distances[data[r][c]] = abs(robot_start[0] - r) + abs(robot_start[1] - c)
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols and (nr, nc) not in visited and data[nr][nc] != '#':
                    visited.add((nr, nc))
                    queue.append((nr, nc))
        return distances

    # Получить все расстояния от каждого робота до ключей
    distances = []
    for robot in robots:
        distances.append(bfs(robot))
        
    # Implementing DP to minimize steps to collect all keys
    dp = {}
    initial_state = tuple(robots), 0  # позиции роботов и битовая маска ключей
    dp[initial_state] = 0
        
    queue = deque([initial_state])
        
    while queue:
        curr_robots, curr_mask = queue.popleft()
        curr_steps = dp[(curr_robots, curr_mask)]
        
        # Пробуйте каждый шаг для каждого робота
        for robot_idx in range(len(robots)):
            for key, pos in key_positions.items():
                if curr_mask & (1 << (ord(key) - ord('a'))):  # Если ключ уже собран, пропустите
                    continue
                
                # Получите расстояние от робота к ключу
                distance = distances[robot_idx].get(key, float('inf'))
                if distance < float('inf'):
                    new_mask = curr_mask | (1 << (ord(key) - ord('a')))
                    new_robots = list(curr_robots)
                    new_robots[robot_idx] = pos  # Обновляем позицию робота
                    new_state = tuple(new_robots), new_mask
                    
                    if new_state not in dp or curr_steps + distance < dp[new_state]:
                        dp[new_state] = curr_steps + distance
                        queue.append(new_state)

    # Находим минимальное количество шагов, когда все ключи собраны
    all_keys_mask = (1 << key_count) - 1
    result = min((steps for (robots_pos, mask), steps in dp.items() if mask == all_keys_mask), default=float('inf'))
    
    return result if result != float('inf') else -1

def main():
    data = get_input()
    result = solve(data)
    print(result)


if __name__ == '__main__':
    main()


0
